# Deribit Api Management

## Api configuration

In [345]:
import pprint
import nest_asyncio
import asyncio
import websockets
import json
import pandas as pd
import datetime as dt
import time
import datetime

ACTUAL_TIME = round(time.time() * 1000) 
# End time is actual time

nest_asyncio.apply()

async def call_api(msg):
   async with websockets.connect('wss://test.deribit.com/ws/api/v2') as websocket:
       await websocket.send(msg)
       while websocket.open:
           response = await websocket.recv()
           return response

def async_loop(api, message):
    return asyncio.get_event_loop().run_until_complete(api(message))


## Simple api calls (getting information)

In [346]:
def get_book_summary(instrument_name):
    msg = \
    {
        "jsonrpc" : "2.0",
        "id" : 3659,
        "method" : "public/get_book_summary_by_instrument",
        "params" : {
            "instrument_name" : instrument_name ,
        }
    }
    resp = async_loop(call_api, json.dumps(msg))

    return resp

def getSummary (currency = 'BTC', type = 'option'):
    msg = \
    {
        "jsonrpc" : "2.0",
        "id" : 9344,
        "method" : "public/get_book_summary_by_currency",
        "params" : {
            "currency" : currency,
            "kind" : type
        }
    }
    resp = async_loop(call_api, json.dumps(msg))

    return resp



    
def epochToDate(epoch):
    return time.strftime('%d-%m-%Y %H:%M',time.localtime(epoch/1000))





## Calling Api

In [347]:
btc_instruments_json_resp = getSummary()
eth_instruments_json_resp = getSummary('ETH')

## Cleaning data

In [348]:
def jsonToDataFrame(json_resp):
    res = json.loads(json_resp)

    df = pd.DataFrame(res['result'])

    return df
    
def historyCleaning(json_resp , drop = False):
    res = json.loads(json_resp)

    df = pd.DataFrame(res['result'])

    df['ticks'] = df.ticks / 1000
    # df['timestamp'] = [dt.datetime.fromtimestamp(date) for date in df.ticks]
    if drop == True:
        df = df[df.volume != 0]

    return df

def dateConverter(date):
    dict = {
        'JAN' : 1 ,
        'FEB' : 2 ,
        'MAR' : 3 ,
        'APR' : 4 ,
        'MAY' : 5 ,
        'JUN' : 6 ,
        'JUL' : 7 ,
        'AUG' : 8 ,
        'SEP' : 9 ,
        'OCT' : 10 ,
        'NOV' : 11 ,
        'DEC' : 12 ,
            
    }
    

    return '{}/{}/{}'.format(date[0:-5], dict[date[-5:-2]], date[-2:])
def dateToEpoch(date):
    a = date.split('/')
    timestamp = datetime.datetime(int('20' + a[2]),int(a[1]),int(a[0]),0,0).timestamp()
    return int(timestamp * 1000)

def epochToDays(epoch):
    if epoch / (1000*60*60*24) < 1:
        return 1
    return epoch / (1000*60*60*24)

def daysPrettyfier(days):
    days = str(round(days,1))
    if days[-1] != 0:
        return "{}d{}h".format(days[0],days[-1])
    
    return days[0] + 'd'
    

def instrumentsCleaning(json_resp, drop=False):
    df = jsonToDataFrame(json_resp)
    print(len(df))
    df = df.sort_values(["instrument_name"]) 
    df['expiry_date'] = [j.split('-')[1] for j in df['instrument_name']]
    
    # df['expiration_timestamp'] = df['expiration_timestamp'].apply(epochToDate)
    df['expiry_date'] = [dateConverter(j) for j in df['expiry_date']]
    df['expiry_date_timestamp'] = df['expiry_date'].apply(dateToEpoch)
    df['expiry_month'] = [j.split("/")[1]  for j in df['expiry_date']]
    df['mark_price_usa'] = round(df['mark_price'] * df['underlying_price'],2)
    df['base'] = [int(j.split("-")[2])  for j in df['instrument_name']]
    df['info'] = (df['mark_price_usa'] + df['base'] - df['underlying_price'] ) / (df['underlying_price'] - df['mark_price_usa'])
    df['underlying_price'] = df['underlying_price'].astype(int)
    
    df['remaining_days'] = df['expiry_date_timestamp']- ACTUAL_TIME
    df['remaining_days'] = df['remaining_days'].apply(epochToDays)

    df['remaining_days_pretty'] = df['remaining_days'].apply(daysPrettyfier)

    df['tasa (%)'] = round((df['info'] / df['remaining_days']) *365,2)
    df['cover'] = round((df['mark_price_usa'] / df['underlying_price']) *100 ,2)
    df['min_loosing_price'] = df['underlying_price'] - (df['underlying_price'] * (df['cover'] / 100))
    df.sort_values(['remaining_days','instrument_name'], inplace=True)

    if drop == True:
        df = df[df.volume != 0]
    
    # df.drop(['base_currency','creation_timestamp','kind','option_type','quote_currency','min_trade_amount','is_active','block_trade_commission'],axis = 1 , inplace=True)
    # "Estimated delivery price"
    df = df[['bid_price','ask_price','high','volume','instrument_name','mark_price_usa','base','underlying_price','tasa (%)','cover','expiry_date','expiry_month','remaining_days_pretty','remaining_days','min_loosing_price','info']]

    return df


In [349]:
btc_instruments_df = instrumentsCleaning(btc_instruments_json_resp, drop=True)
eth_instruments_df = instrumentsCleaning(eth_instruments_json_resp, drop=True)


228
264


In [350]:
def separateCallPut(df):

    df['optionType'] = [j[-1] for j in df['instrument_name']]
    # instrument_df['expiry_date'] = [j[5:-8] for j in instrument_df['expiry_date']]

    put_df = df[df['optionType'] == 'P'].drop('optionType',axis=1)

    put_df.drop(['tasa (%)','base'],axis=1,inplace=True)

    call_df = df[df['optionType'] == 'C'].drop('optionType',axis=1)

    

    return call_df, put_df

# put_df.set_index('instrument_name', inplace=True)
# call_df.set_index('instrument_name', inplace=True)

btc_call_df,btc_put_df = separateCallPut(btc_instruments_df)
eth_call_df,eth_put_df = separateCallPut(eth_instruments_df)

In [351]:
def separateCallsByDate(df):
    dataframes = []
    for region, df_region in df.groupby('remaining_days'):
        dataframes.append([region,df_region])

    return dataframes

btc_dataframes = separateCallsByDate(btc_call_df)
eth_dataframes = separateCallsByDate(eth_call_df)




## Eporting to Excel

In [352]:
# from openpyxl import load_workbook
 
# file_name = 'nada.xlsx'
 
# #load excel file
# workbook = load_workbook(filename= 'C:/Users/Alpha/My Drive/codingProjects/deribit/' + file_name)
 
# #open workbook
# sheet = workbook.active
# # rangeExcel = 1
# # #modify the desired cell
# # acumRows = 1
# # for j in range(1,len(dataframes)):
# #     sheet["C" + str(acumRows)] = dataframes[j][0]
# #     sheet["B" + str(acumRows + 1)] = 'Base'
# #     sheet["C" + str(acumRows + 1)] = 'Prima'
# #     sheet["D" + str(acumRows + 1)] = 'Tasa'
# #     sheet["E" + str(acumRows + 1)] = 'Cobertura'

# #     sheet["A" + str(acumRows + 2)] = dataframes[j][1]

# #     sheet["D4"] = '=D17+$C$2'
    
# #     acumRows = len(dataframes[j][1] + 3)
# sheet["A1"] = call_df['instrument_name']

# #save the file
# workbook.save(filename='C:/Users/Alpha/My Drive/codingProjects/deribit/' + file_name)

In [353]:
file_path = 'C:/Users/Alpha/My Drive/codingProjects/deribit/'


def paraDario(df, underlying='BTC'):
    df['prima'] = df['mark_price_usa']
    df['precio_' + underlying + '_actual'] = df['underlying_price']
    excel_call_df = df[['info','instrument_name','volume','prima','tasa (%)','cover','min_loosing_price','precio_' + underlying + '_actual','expiry_date','remaining_days_pretty']]
    return excel_call_df
# saving the excel
excel_btc_call_df = paraDario(btc_call_df)
excel_eth_call_df = paraDario(eth_call_df, 'ETH')

excel_btc_call_df.to_excel(file_path + 'BTCOptions.xlsx', index=False)
excel_eth_call_df.to_excel(file_path + 'ETHOptions.xlsx', index=False)

print('Succesful write')

Succesful write
